<a href="https://colab.research.google.com/github/avilum/audio2text/blob/master/wav2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers librosa torch

In [ ]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [ ]:
#load model and tokenizer
_model = "facebook/wav2vec2-large-960h"
tokenizer = Wav2Vec2Tokenizer.from_pretrained(_model)
model = Wav2Vec2ForCTC.from_pretrained(_model)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:746: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#load any audio file of your choice
# speech, rate = librosa.load("~/Desktop/hi.wav",sr=8000)
speech, rate = librosa.load("Desktop/hi.m4a",sr=16000) # SHOULD BE 'HAMBURGER WITH BACON'...

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [ ]:
print('Speech:', speech)
print('Shape:', speech.shape)

Speech: [0.00128225 0.00222278 0.00200444 ... 0.02562797 0.03500057 0.        ]
Shape: (49131,)


In [ ]:
import IPython.display as ipd
ipd.Audio(speech, rate=rate)

In [ ]:
# Preprocessing
input_values = tokenizer(speech, return_tensors = 'pt').input_values

# Inference
# Store logits (non-normalized predictions)
logits = model(input_values).logits
logits.shape

torch.Size([1, 153, 32])

In [ ]:
# Postprocessing - Getting the predicted vectors words id's in the speech array
predicted_ids = torch.argmax(logits, dim =-1)

# Decoding the audio to generate text (length derived from the sampling rate)
transcriptions = tokenizer.decode(predicted_ids[0])
print(transcriptions)

HEMBURGER WITH BACON


In [ ]:
predicted_ids[0].shape

torch.Size([153])

In [ ]:
# E2E simplified:
_model = "facebook/wav2vec2-large-960h"
tokenizer = Wav2Vec2Tokenizer.from_pretrained(_model)
model = Wav2Vec2ForCTC.from_pretrained(_model)
processor = Wav2Vec2Processor.from_pretrained(_model)

# Load the file for inference
speech, rate = librosa.load("Desktop/hi.m4a",sr=16000)
input_values = processor(speech, return_tensors="pt", padding="longest").input_values  # Batch size 1
 
# retrieve logits (inference)
logits = model(input_values).logits

# take argmax and decode (postprocessing)
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

# Print the text
print(transcription)

# continue with your app... :) 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:746: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/librosa/util

['HEMBURGER WITH BACON']


In [ ]:
# Second inference is much faster...
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(transcription)

['HEMBURGER WITH BACON']
